In [1]:
import ctypes
from ctypes import wintypes as wt
from ctypes import sizeof
from time import sleep, time

In [2]:
INPUT_MOUSE = 0
INPUT_KEYBOARD = 1
INPUT_HARDWARE = 2

MOUSEEVENTF_ABSOLUTE = 0x8000
MOUSEEVENTF_LEFTDOWN = 0x0002 
MOUSEEVENTF_LEFTUP = 0x0004
MOUSEEVENTF_MOVE = 0x0001
MOUSEEVENTF_RIGHTDOWN = 0x0008
MOUSEEVENTF_RIGHTUP = 0x0010
MOUSEEVENTF_VIRTUALDESK = 0x4000

In [3]:
ULONG_PTR = ctypes.POINTER(ctypes.c_ulong)
LONG = ctypes.c_long

# These depend on whether UNICODE is defined
LPCTSTR = wt.LPCSTR  # LPCWSTR or LPCSTR
LPTSTR = wt.LPSTR  # LPWSTR or LPSTR

WNDENUMPROC = ctypes.WINFUNCTYPE(wt.BOOL, wt.HWND, wt.LPARAM)

In [4]:
class KEYBDINPUT(ctypes.Structure):
    _fields_ = [
        ('wVk', wt.WORD),
        ('wScan', wt.WORD),
        ("dwFlags", wt.DWORD),
        ("time", wt.DWORD),
        ("dwextraInfo", ULONG_PTR)
    ]
    
class MOUSEINPUT(ctypes.Structure):
    _fields_ = [
        ('dx', LONG),
        ('dy', LONG),
        ('mouseData', wt.DWORD),
        ('dwFlags', wt.DWORD),
        ('time', wt.DWORD),
        ('dwExtraInfo', ULONG_PTR),
    ]
    
class HARDWAREINPUT(ctypes.Structure):
    _fields_ = [
        ('uMsg', wt.DWORD),
        ('wParamL', wt.WORD),
        ('wParamH', wt.WORD)
    ]
    

In [5]:
class InptUnion(ctypes.Union):
    _fields_ = [
        ('mi', MOUSEINPUT),
        ('ki', KEYBDINPUT),
        ('hi', HARDWAREINPUT)
    ]

class INPUT(ctypes.Structure):
    _fields_ = [
        ('type', wt.DWORD),
        ('ip', InptUnion)
    ]
    
PINPUT = ctypes.POINTER(INPUT)

In [6]:
def make_kb_input(keycode, scan_code=0, flags=0):
    ip = INPUT()
    ip.type = INPUT_KEYBOARD
    
    ki = KEYBDINPUT()
    ki.wVk = keycode
    ki.wScan = scan_code
    ki.dwFlags = flags
    
    ip.ip.ki = ki
    return ip    

In [7]:
# Raw exported windows API functions

user32 = ctypes.windll.user32
k32 = ctypes.windll.kernel32

_SendInput = user32.SendInput
_SendInput.argtypes = [wt.UINT, PINPUT, ctypes.c_int]
_SendInput.restype = wt.UINT

def SendInput(*args):
    res = _SendInput(*args)
    if not res:
        raise OSError(k32.GetLastError())
    return res

FindWindow = user32.FindWindowA
FindWindow.argtypes = [LPCTSTR, LPCTSTR]
FindWindow.restype = wt.HWND

CloseHandle = k32.CloseHandle
CloseHandle.argtypes = [wt.HANDLE]
CloseHandle.restype = wt.BOOL

EnumWindows = user32.EnumWindows
EnumWindows.argtypes = [WNDENUMPROC, wt.LPARAM]
EnumWindows.restype = wt.BOOL

GetWindowText = user32.GetWindowTextA
GetWindowText.argtypes = [wt.HWND, LPTSTR, ctypes.c_int]
GetWindowText.restype = ctypes.c_int

SetActiveWindow = user32.SetActiveWindow
SetActiveWindow.argtypes = [wt.HWND]
SetActiveWindow.restype = wt.HWND

SetForegroundWindow = user32.SetForegroundWindow
SetForegroundWindow.argtypes = [wt.HWND]
SetForegroundWindow.restype = wt.BOOL

GetLastError = k32.GetLastError
GetLastError.argtypes = []
GetLastError.restype = ctypes.c_int

GetForegroundWindow = user32.GetForegroundWindow
GetForegroundWindow.argtypes = []
GetForegroundWindow.restype = wt.HWND

AttachThreadInput = user32.AttachThreadInput
AttachThreadInput.argtypes = [wt.DWORD, wt.DWORD, wt.BOOL]
AttachThreadInput.restype = wt.BOOL

In [8]:
def get_window_text(hwnd):
    sz = 256
    buf = ctypes.create_string_buffer(sz)
    r = GetWindowText(hwnd, buf, sz)
    if not r:
        return ""
    else:
        return buf.value.decode('ascii')

def _windows_cb(hwnd, lparam):
    """ Example callback for EnumWindows()"""
    sz = 100
    buf = ctypes.create_string_buffer(sz)
    r = GetWindowText(hwnd, buf, sz)
    if r == 0:
        e = GetLastError()
        if e:
            print("RV was 0:", GetLastError())
    else:
        print(buf.value)
    # return 1 to continue iteration
    # return 0 to stop iteration
    return 1

windows_cb = WNDENUMPROC(_windows_cb)

def enum_windows():
    EnumWindows(windows_cb, 0)

In [9]:
def make_mouse_move(x,y):
    ip = INPUT()
    ip.type = INPUT_MOUSE
    
    mi = MOUSEINPUT()
    mi.dx = x
    mi.dy = y
    mi.mouseData = 0
    mi.time = 0
    mi.dwFlags=(MOUSEEVENTF_MOVE) | MOUSEEVENTF_ABSOLUTE | MOUSEEVENTF_VIRTUALDESK
    ip.ip.mi = mi
    return ip

In [10]:
def make_mouse_left_click(dwFlags):
    ip = INPUT()
    ip.type = INPUT_MOUSE
    
    mi = MOUSEINPUT()
    mi.dx = 0
    mi.dy = 0
    mi.mouseData = 0
    mi.time = 0
    mi.dwFlags = MOUSEEVENTF_ABSOLUTE | MOUSEEVENTF_VIRTUALDESK | dwFlags
    ip.ip.mi = mi
    return ip

In [11]:
import win32api
SCREEN_X = win32api.GetSystemMetrics(78)
SCREEN_Y = win32api.GetSystemMetrics(79)

def norm(p):
    return int(p.x / (SCREEN_X) * 65535), int(p.y / (SCREEN_Y) * 65536)

def norm2(p):
    return int(p[0] / (SCREEN_X) * 65535), int(p[1] / (SCREEN_Y) * 65536)

In [12]:
def send_keyboard_input(keycode, scan_code=0, flags=0):
    ip = make_kb_input(keycode, scan_code, flags)
    return SendInput(1, PINPUT(ip), ctypes.sizeof(ip))

def mouse_move(x,y):
    ip = make_mouse_move(x,y)
    return SendInput(1, PINPUT(ip), ctypes.sizeof(ip))
    
def mouse_click():
    #ip = make_mouse_left_click(x,y, MOUSEEVENTF_LEFTDOWN)
    #SendInput(1, PINPUT(ip), ctypes.sizeof(ip))
    #ip = make_mouse_left_click(x, y, MOUSEEVENTF_LEFTUP)
    
    ip2 = make_mouse_left_click(MOUSEEVENTF_LEFTDOWN)
    #SendInput(1, PINPUT(ip), ctypes.sizeof(ip))
    ip3 = make_mouse_left_click(MOUSEEVENTF_LEFTUP)
    #SendInput(1, PINPUT(ip), ctypes.sizeof(ip))
    send_inputs(ip2, ip3)

def send_inputs(*ips):
    typ = ips[0].type
    sz = ctypes.sizeof(ips[0])
    assert all(ip.type == typ for ip in ips)
    assert all(ctypes.sizeof(ip) == sz for ip in ips)
    array = (INPUT * len(ips))()
    for i, ip in enumerate(ips):
        array[i] = ip
    return SendInput(len(ips), array, sz)

In [13]:
def get_pos():
    point = ctypes.wintypes.POINT()
    ctypes.windll.user32.GetCursorPos(ctypes.byref(point))
    x, y = norm(point)
    return x,y

def get_pos2():
    point = ctypes.wintypes.POINT()
    ctypes.windll.user32.GetCursorPos(ctypes.byref(point))
    return point.x, point.y

In [14]:
def run_clicks(wait=0.05):
    xs, ys = get_pos()
    try:
        while True:
            mouse_click()
            sleep(wait)
        #sleep(5)
            xs2, xy2 = get_pos()
            if xs2 != xs or xy2 != ys:
                return
    except KeyboardInterrupt:
        pass

def run_clicks2(x, y, wait=0.05):
    while True:
        x1, y1 = get_pos()
        mouse_move(x, y)
        mouse_click()
        mouse_move(x1, y1)
        sleep(wait)
        if win32api.GetKeyState(0x02) & 0x8000:
            return
        

In [15]:
# def right_click_down():
#     return win32api.GetKeyState(0x02) & 0x8000

# def click_for(t, w=0.05):
#     n = time()
#     e = n + t
#     x,y = get_pos()
#     while time() < e:
#         mouse_click()
#         sleep(w)
#         x2, y2 = get_pos()
#         if x != x2 or y != y2 or right_click_down():
#             raise StopIteration

# def mouse_move2(p):
#     x,y = p; mouse_move(x,y)
    
# c2a = 50897, 31602
# gh = 50681, 38666
# hl = 50681, 34952
# mid = 44365, 33132

# while True:
#     if right_click_down():
#         break
#     mouse_move2(c2a)
#     mouse_click()
    
#     mouse_move2(gh)
#     mouse_click()
    
#     mouse_move2(mid)
#     click_for(16)
    
#     mouse_move2(hl)
#     mouse_click()
    
#     mouse_move2(mid)
#     click_for(21)

In [16]:
def get_cursor_hwnd():
    return win32gui.WindowFromPoint(get_pos2())

In [17]:
class NotFound(NameError): pass
def find_window(title):
    if not isinstance(title, bytes):
        title = title.encode('ascii')
    h = FindWindow(None, title)
    if not h:
        raise NameError(title)
    return h


In [18]:
def is_foreground(hcmp):
    hfgd = GetForegroundWindow()
    if hfgd == hcmp:
        rv = True
    else:
        rv = False
    return rv

In [19]:
from time import sleep
import tkinter as tk
import tkinter.ttk as ttk
import win32gui
import os

In [20]:
SK_X = 0
SK_Y = 0

def get_coord():
    global SK_X, SK_Y
    sleep(2)
    SK_X, SK_Y = get_pos()
    print("pos got!")
    
def do_click(wait):
    x, y = get_pos()
    mouse_move(SK_X, SK_Y)
    run_clicks(wait)
    mouse_move(x, y)
    
def set_topmost(hwnd):
    GWL_EXSTYLE = -20
    WS_EX_TOPMOST = 0x00000008
    dwExStyle = win32gui.GetWindowLong(hwnd, GWL_EXSTYLE);
    dwExStyle |= WS_EX_TOPMOST;
    win32gui.SetWindowLong(hwnd, GWL_EXSTYLE, dwExStyle);
    
def allow_set_foreground():
    if not ctypes.windll.user32.AllowSetForegroundWindow(os.getpid()):
        raise OSError(win32api.GetLastError())
    
def sendkeys(wait=0.05):
    r = tk.Tk()
    f = ttk.LabelFrame(r, text="click")
    coord = ttk.Button(f, text="Get Pos", command=get_coord)
    click = ttk.Button(f, text="Click", command=lambda: do_click(wait))
    stop  = ttk.Button(f, text="Stop", command=lambda: r.destroy())
   
    f.grid()
    coord.grid(row=1, column=0)
    click.grid(row=1, column=1)
    stop.grid(row=1, column=2)
    set_topmost(r.winfo_id())
    
    r.mainloop()

In [21]:
sendkeys(0.05)

pos got!


In [ ]:
def set_topmost(hwnd):
    GWL_EXSTYLE = -20
    GWL_EXSTYLE = 0x00000008L
    dwExStyle = win32gui.GetWindowLong(hwnd, GWL_EXSTYLE);
    dwExStyle &= ~WS_EX_TOPMOST;
    win32gui.SetWindowLong(hwnd, GWL_EXSTYLE, dwExStyle);

In [18]:
class SendKeysClient():
    def __init__(self, hwnd=0):
        if isinstance(hwnd, int):
            self.hwnd = hwnd
        elif isinstance(hwnd, (str, bytes)):
            self.hwnd = find_window(hwnd)
        else:
            raise TypeError(type(hwnd))
                
    def is_foreground(self):
        return is_foreground(self.hwnd)
    
    def get_window_name(self):
        return get_window_text(self.hwnd)
    
    def send_key(self, key):
        while not self.is_foreground():
            sleep(0.1)
        send_keyboard_input(key)
        
    def __del__(self):
        if self.hwnd:
            CloseHandle(self.hwnd)
  

In [19]:
class SimpleButton(ttk.Button):
    def __init__(self, parent, text, cmd):
        super().__init__(parent, text=text, command=cmd)
        
class SimpleLabel(ttk.Label):
    def __init__(self, parent, text):
        self.var = tk.StringVar(None, text)
        super().__init__(parent, textvariable=self.var)
        
    def get(self):
        return self.var.get()
    
    def set(self, s):
        self.var.set(s)
        

class _StatusFrameField():
    def __init__(self, parent, name, value=""):
        self.l_name = SimpleLabel(parent, text=name)
        self.l_value = SimpleLabel(parent, text=value)
    
    def grid(self, row=0, col=0):
        sticky_WN = (tk.W, tk.N)
        self.l_name.grid(row=row, column=col, sticky=sticky_WN)
        self.l_value.grid(row=row, column=col+1, sticky=sticky_WN)
        
    def grid_forget(self):
        self.frame.grid_forget()
        
    def set_value(self, v):
        self.l_value.set(str(v))
        
    def set_name(self, n):
        self.l_name.set(n)
        
        
class StatusFrame():
    def __init__(self, parent, title="StatusFrame", fields=()):
        self.frame = ttk.LabelFrame(parent, text=title)
        self.field_labels = {}
        for f in fields:
            self.add_field(f)
    
    def add_field(self, key, text, value=""):
        f = _StatusFrameField(self.frame, text, value)
        self.field_labels[key] = f
        return f
        
    def grid(self, row=0, column=0, **kw):
        rows = len(self.field_labels) or 1
        self.frame.grid(row=row, column=column, rowspan=rows, columnspan=2, **kw)
        for i, f in enumerate(self.field_labels.values()):
            f.grid(i)
            
    def __getitem__(self, key):
        return self.field_labels[key]
    
    def update_field(self, key, text=None, value=None):
        f = self.field_labels[key]
        if text:
            f.l_name.set(text)
        if value:
            f.l_value.set(value)
        
    def update_field_value(self, key, value):
        self.update_field(key, None, value)
    
    def update_field_text(self, key, text):
        self.update_field(key, text, None)
                

In [20]:
class SendKeysFrame():
    def __init__(self, parent, text):
        self.frame = ttk.LabelFrame(parent, text=text)
        self.expl_label = SimpleLabel(self.frame, text="")
        self.start = SimpleButton(self.frame, "Start", lambda: self.start_cmd())
        self.stop = SimpleButton(self.frame, "Stop", lambda: self.stop_cmd())
        self.status = StatusFrame(parent, "Status:")
        
    def start_cmd(self):
        pass
    
    def stop_cmd(self):
        pass
    
    def grid(self, row=0, col=0):
        self.frame.grid(row=row, column=col, sticky=tk.W)
        self.start.grid(row=0, column=0)
        self.stop.grid(row=0, column=1)
        self.status.grid(row=row+1, column=0)
        
    def grid_forget(self):
        self.frame.grid_forget()
        self.start.grid_forget()
        self.stop.grid_forget()
        
    def set_explain(self, e):
        self.expl_label.set(e)
        
    def add_status(self, name, val=""):
        return self.status.add_field(name, val)
    

In [21]:
def test1():
    r = tk.Tk()
    f = ttk.LabelFrame(r, text="My Frame")
    start = ttk.Button(f, text="Start")
    stop = ttk.Button(f, text="Stop", command=lambda: r.destroy())
    label = ttk.Label(f, text="Send key presses to application")
    status = StatusFrame(f)
    status.add_field("foo", "bar")
    status.add_field("baz", "hi")
    status.add_field("hello", "world")

    f.grid()
    label.grid(row=0, column=0, columnspan=2)
    start.grid(row=1, column=0)
    stop.grid(row=1, column=1)
    status.grid(2, 0)
    status['foo'].set_value("foobar")

    r.mainloop()

In [22]:
class DiabloMacroApp():
    def __init__(self, hwnd=0):
        self.client = DiabloKeysClient(hwnd)
        self.root = tk.Tk()
        self.frame = SendKeysFrame(self.root, "Diablo Macro App")
        self.frame.start_cmd = self.start
        self.frame.stop_cmd = self.stop
        
        self.update_counter = 0
        self.hbt_f = self.frame.add_status("Update Counter:", 0)
        self.wtitle_f = self.frame.add_status("Window:", self.client.get_window_name())
        self.hwnd_f = self.frame.add_status("hWnd:", self.client.hwnd)
        self.fgd_f = self.frame.add_status("In Foreground:", False)
        self.debug_f = self.frame.add_status("Debug_Info:", "")
        self.debug_f.counter = 0
        
        # Misc
        self.root.bind("<Visibility>", self.event_handler)
        self.root.bind("<Map>", self.event_handler)
        self.root.bind("")
        
        # tkinter handling
        self._update_id = None
        self._update_interval = 100
        
    def event_handler(self, e):
        self.debug_f.counter += 1
        s = "%s\n%s\n%s" % (self.debug_f.counter, str(e), fmt_dir(e))
        self.debug_f.set_value(s)

    def run(self):
        self.schedule_update()
        self.frame.grid()
        self.root.lift()
        self.root.mainloop()
        
    def start(self):
        pass
        
    def stop(self):
        pass
        
    def schedule_update(self):
        try:
            self._update()
        finally:
            self._update_id = self.root.after(self._update_interval, self.schedule_update)
            
    def _update(self):
        self.update_counter += 1
        self.hbt_f.set_value(self.update_counter)
        
        title = self.client.get_window_name()
        self.wtitle_f.set_value(title)
        
        in_fg = self.client.is_foreground()
        self.fgd_f.set_value(in_fg)

In [23]:
def test2(title='Diablo III'):
    d = DiabloMacroApp(hwnd=find_window(title))
    d.run()
    return d

In [24]:
def find_npp():
    h = 0
    def find_wnd_cb(hwnd, lparam):
        nonlocal h
        wnd_frag = ctypes.cast(lparam, py_object_p).contents.value
        if wnd_frag in get_window_text(hwnd):
            h = hwnd
            return 0
        return 1
    find_npp = WNDENUMPROC(_find_npp)
    py_object_p = ctypes.POINTER(ctypes.py_object)
    foo = ctypes.py_object("Notepad++")
    foo = py_object_p(foo)
    lp = ctypes.cast(foo, ctypes.c_void_p)
    EnumWindows(find_npp, wt.LPARAM(lp.value))
    assert h, "npp not running"
    return h

In [25]:
import types

def trunc(s, n=20):
    return str(s)[:20]

def is_magic(a): return a.startswith("__") and a.endswith("__")
def is_func(v): return isinstance(v, (types.MethodType, types.FunctionType))
def is_private(a): return a.startswith("_")
    
def fmt_dir(o, sep='\n', skip_private=False, skip_functions=True, skip_magic=True):
    attrs = dir(o)
    buf = []
    for a in attrs:
        if skip_private and is_private(a): continue
        if skip_magic and is_magic(a): continue
        
        try:
            v = getattr(o, a)
        except AttributeError:
            v = "<error>"
            
        if skip_functions and is_func(v): continue
        
        buf.append("%s: %s" % (trunc(a), trunc(v)))
    return sep.join(buf)    

In [32]:
WM_LBUTTONDOWN = 0x0201
WM_LBUTTONUP = 0x0202
x2, y2 = norm2((x, y))
win32api.SendMessage(4196156, WM_LBUTTONDOWN, 0x0001, pack_coord(x2, y2))
win32api.SendMessage(4196156, WM_LBUTTONUP, 0, pack_coord(x2, y2))

def pack_coord(x, y):
    return ((y & 0xffff) << 16) | (x & 0xffff)test1()